In [96]:

# import all libraries and dependencies for dataframe
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime, timedelta
from wwo_hist import retrieve_hist_data
import pickle

import csv
import codecs
import urllib.request
import sys

import requests
import json

# import all libraries and dependencies for data visualization
pd.options.display.float_format='{:.4f}'.format
plt.rcParams['figure.figsize'] = [8,8]
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1) 
sns.set(style='darkgrid')

In [52]:
api_key = 'xxxxxxxxxxxxxxxxxx'

In [111]:
df = pd.read_csv('hotels_mod1.csv')
#`arrival_date_month` names converted to the numbers

df['arrival_date_month'].replace({'January' : '01',
        'February' : '02',
        'March' : '03',
        'April' : '04',
        'May' : '05',
        'June' : '06',
        'July' : '07',
        'August' : '08',
        'September' : '09', 
        'October' : '10',
        'November' : '11',
        'December' : '12'}, inplace=True)
df['arrival_date_day_of_month'] = df['arrival_date_day_of_month'].astype(str)
df['arrival_date_month'] = df['arrival_date_month'].astype(str)
df['arrival_date_year'] = df['arrival_date_year'].astype(str)

df['arrival_date_day_of_month'].replace({'1' : '01',
        '2' : '02',
        '3' : '03',
        '4' : '04',
        '5' : '05',
        '6' : '06',
        '7' : '07',
        '8' : '08',
        '9' : '09', 
        }, inplace=True)
df['date'] = df['arrival_date_year'] + '-' + df['arrival_date_month'] + '-' + df['arrival_date_day_of_month']

df.shape

(118984, 40)

In [112]:
df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,used_agent,international_booking,deposit_type1,family_combo,arrival_date_season,reassigned,Stay_length,date
0,Resort Hotel,0,342,2015,07,27,01,0,0,2,0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,nan,nan,0,Transient,0.0000,0,0,Check-Out,2015-07-01,False,False,No Deposit,Adults only,Summer,False,0,2015-07-01
1,Resort Hotel,0,737,2015,07,27,01,0,0,2,0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,nan,nan,0,Transient,0.0000,0,0,Check-Out,2015-07-01,False,False,No Deposit,Adults only,Summer,False,0,2015-07-01
2,Resort Hotel,0,7,2015,07,27,01,0,1,1,0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,nan,nan,0,Transient,75.0000,0,0,Check-Out,2015-07-02,False,True,No Deposit,Adults only,Summer,True,1,2015-07-01
3,Resort Hotel,0,13,2015,07,27,01,0,1,1,0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.0000,nan,0,Transient,75.0000,0,0,Check-Out,2015-07-02,True,True,No Deposit,Adults only,Summer,False,1,2015-07-01
4,Resort Hotel,0,14,2015,07,27,01,0,2,2,0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0000,nan,0,Transient,98.0000,0,1,Check-Out,2015-07-03,True,True,No Deposit,Adults only,Summer,False,2,2015-07-01


In [20]:
print(df['date'].nunique())
df['date'].value_counts()

793


2015-12-5     448
2016-11-7     365
2015-10-16    355
2016-10-13    344
2015-9-18     340
             ... 
2015-12-15    29 
2015-11-15    28 
2015-12-7     27 
2015-11-29    20 
2015-12-13    18 
Name: date, Length: 793, dtype: int64

In [27]:
df.groupby(['hotel', 'date']).size()

hotel         date      
City Hotel    2015-10-1     78 
              2015-10-10    173
              2015-10-11    59 
              2015-10-12    132
              2015-10-13    63 
                            .. 
Resort Hotel  2017-8-5      69 
              2017-8-6      74 
              2017-8-7      84 
              2017-8-8      48 
              2017-8-9      32 
Length: 1586, dtype: int64

In [92]:
city_date = df.loc[df['hotel'] == 'City Hotel']['date'].unique().tolist()
resort_date = df.loc[df['hotel'] == 'Resort Hotel']['date'].unique().tolist()
all_date = df['date'].unique().tolist()
all_date.sort()
resort_date.sort()
city_date.sort()
#resort_date

In [87]:
# This is the core of our weather query URL
BaseURL = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/'

# Set up the location parameter for our query
QueryLocation = '&locations=Faro%2C%20Portugal%7CLisbon%2C%20Portugal'

# Set up the query type parameter for our query ('FORECAST' or 'HISTORY')
QueryType= 'HISTORY'

# Set up the key parameter for our query
QueryKey = '&key=' + api_key

# # Set up the date parameters for our query. Used only for historical weather data requests
# FromDateParam = sys.argv[4]
# ToDateParam = sys.argv[5]


In [127]:

# History requests require a date.  We use the same date for start and end since we only want to query a single date in this example
QueryStartDate = '&startDateTime=' + '2015-07-01' + 'T00:00:00'
QueryEndDate = '&endDateTime=' + '2017-08-31' + 'T00:00:00'

QueryTypeParams = 'history?&aggregateHours=24&unitGroup=us&dayStartTime=0:0:00&dayEndTime=0:0:00' + QueryStartDate + QueryEndDate

QueryContentType = '&contentType=' + 'json'

# Build the entire query
URL = BaseURL + QueryTypeParams + QueryLocation + QueryContentType + QueryKey

print(' - Running query URL: ', URL)
print()

# Parse the results as CSV
pull_data = requests.get(URL)
# CSVText = csv.reader(codecs.iterdecode(CSVBytes, 'utf-8'))

 - Running query URL:  https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/history?&aggregateHours=24&unitGroup=us&dayStartTime=0:0:00&dayEndTime=0:0:00&startDateTime=2015-07-01T00:00:00&endDateTime=2017-08-31T00:00:00&locations=Faro%2C%20Portugal%7CLisbon%2C%20Portugal&contentType=json&key=LP4ZMFA846TAAFSVHEQCMYJRY



In [138]:
filename = 'weather'
outfile = open(filename, 'wb')
pickle.dump(json_data, outfile)
outfile.close


<function BufferedWriter.close>

In [130]:
json_data = pull_data.json()
city_weather = pd.DataFrame.from_dict(json_data['locations']['Lisbon, Portugal']['values'])
city_weather['hotel'] = 'City Hotel'
city_weather['date'] = [x[0:10] for x in city_weather['datetimeStr']]
resort_weather = pd.DataFrame.from_dict(json_data['locations']['Faro, Portugal']['values'])
resort_weather['hotel'] = 'Resort Hotel'
resort_weather['date'] = [x[0:10] for x in resort_weather['datetimeStr']]
resort_weather.head()

,wdir,temp,maxt,visibility,wspd,datetimeStr,solarenergy,heatindex,cloudcover,mint,datetime,precip,solarradiation,weathertype,snowdepth,sealevelpressure,dew,humidity,precipcover,wgust,conditions,windchill,info,hotel,date
0,226.6700,70.1000,77.0000,6.2000,23.0000,2015-07-01T00:00:00+01:00,None,nan,18.6000,62.6000,1435708800000,0.0000,None,None,nan,nan,62.5000,77.3400,0.0000,nan,Clear,nan,None,Resort Hotel,2015-07-01
1,283.4800,73.3000,80.6000,6.2000,16.1000,2015-07-02T00:00:00+01:00,None,80.1000,0.0000,64.4000,1435795200000,0.0000,None,None,nan,nan,58.1000,60.6000,0.0000,nan,Clear,nan,None,Resort Hotel,2015-07-02
2,203.3300,74.5000,80.6000,6.2000,15.0000,2015-07-03T00:00:00+01:00,None,81.1000,0.0000,66.2000,1435881600000,0.0000,None,None,nan,nan,54.4000,50.2400,0.0000,nan,Clear,nan,None,Resort Hotel,2015-07-03
3,235.5000,75.7000,84.2000,6.2000,12.8000,2015-07-04T00:00:00+01:00,None,85.7000,0.0000,66.2000,1435968000000,0.0000,None,None,nan,nan,54.7000,48.9600,0.0000,nan,Clear,nan,None,Resort Hotel,2015-07-04
4,219.1300,80.5000,84.2000,6.2000,12.8000,2015-07-05T00:00:00+01:00,None,85.1000,0.0000,77.0000,1436054400000,0.0000,None,None,nan,nan,61.3000,52.6400,0.0000,nan,Clear,nan,None,Resort Hotel,2015-07-05


In [131]:
all_weather = pd.concat([city_weather, resort_weather])
all_weather.shape
#all_weather['date'] = all_weather['datetimeStr'][0][0:10]

(1586, 25)

In [132]:
df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,used_agent,international_booking,deposit_type1,family_combo,arrival_date_season,reassigned,Stay_length,date
0,Resort Hotel,0,342,2015,07,27,01,0,0,2,0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,nan,nan,0,Transient,0.0000,0,0,Check-Out,2015-07-01,False,False,No Deposit,Adults only,Summer,False,0,2015-07-01
1,Resort Hotel,0,737,2015,07,27,01,0,0,2,0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,nan,nan,0,Transient,0.0000,0,0,Check-Out,2015-07-01,False,False,No Deposit,Adults only,Summer,False,0,2015-07-01
2,Resort Hotel,0,7,2015,07,27,01,0,1,1,0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,nan,nan,0,Transient,75.0000,0,0,Check-Out,2015-07-02,False,True,No Deposit,Adults only,Summer,True,1,2015-07-01
3,Resort Hotel,0,13,2015,07,27,01,0,1,1,0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.0000,nan,0,Transient,75.0000,0,0,Check-Out,2015-07-02,True,True,No Deposit,Adults only,Summer,False,1,2015-07-01
4,Resort Hotel,0,14,2015,07,27,01,0,2,2,0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0000,nan,0,Transient,98.0000,0,1,Check-Out,2015-07-03,True,True,No Deposit,Adults only,Summer,False,2,2015-07-01


In [143]:
final_df = pd.merge(left = df, right = all_weather, how = 'left', on = ['hotel', 'date'])
# final_df = pd.merge(left = final_df, right = resort_weather, how = 'left', on = ['hotel', 'date'])
final_df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,used_agent,international_booking,deposit_type1,family_combo,arrival_date_season,reassigned,Stay_length,date,wdir,temp,maxt,visibility,wspd,datetimeStr,solarenergy,heatindex,cloudcover,mint,datetime,precip,solarradiation,weathertype,snowdepth,sealevelpressure,dew,humidity,precipcover,wgust,conditions,windchill,info
0,Resort Hotel,0,342,2015,07,27,01,0,0,2,0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,nan,nan,0,Transient,0.0000,0,0,Check-Out,2015-07-01,False,False,No Deposit,Adults only,Summer,False,0,2015-07-01,226.6700,70.1000,77.0000,6.2000,23.0000,2015-07-01T00:00:00+01:00,None,nan,18.6000,62.6000,1435708800000,0.0000,None,None,nan,nan,62.5000,77.3400,0.0000,nan,Clear,nan,None
1,Resort Hotel,0,737,2015,07,27,01,0,0,2,0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,nan,nan,0,Transient,0.0000,0,0,Check-Out,2015-07-01,False,False,No Deposit,Adults only,Summer,False,0,2015-07-01,226.6700,70.1000,77.0000,6.2000,23.0000,2015-07-01T00:00:00+01:00,None,nan,18.6000,62.6000,1435708800000,0.0000,None,None,nan,nan,62.5000,77.3400,0.0000,nan,Clear,nan,None
2,Resort Hotel,0,7,2015,07,27,01,0,1,1,0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,nan,nan,0,Transient,75.0000,0,0,Check-Out,2015-07-02,False,True,No Deposit,Adults only,Summer,True,1,2015-07-01,226.6700,70.1000,77.0000,6.2000,23.0000,2015-07-01T00:00:00+01:00,None,nan,18.6000,62.6000,1435708800000,0.0000,None,None,nan,nan,62.5000,77.3400,0.0000,nan,Clear,nan,None
3,Resort Hotel,0,13,2015,07,27,01,0,1,1,0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.0000,nan,0,Transient,75.0000,0,0,Check-Out,2015-07-02,True,True,No Deposit,Adults only,Summer,False,1,2015-07-01,226.6700,70.1000,77.0000,6.2000,23.0000,2015-07-01T00:00:00+01:00,None,nan,18.6000,62.6000,1435708800000,0.0000,None,None,nan,nan,62.5000,77.3400,0.0000,nan,Clear,nan,None
4,Resort Hotel,0,14,2015,07,27,01,0,2,2,0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0000,nan,0,Transient,98.0000,0,1,Check-Out,2015-07-03,True,True,No Deposit,Adults only,Summer,False,2,2015-07-01,226.6700,70.1000,77.0000,6.2000,23.0000,2015-07-01T00:00:00+01:00,None,nan,18.6000,62.6000,1435708800000,0.0000,None,None,nan,nan,62.5000,77.3400,0.0000,nan,Clear,nan,None


In [139]:
final_df.shape

(118984, 63)

In [140]:
df.shape

(118984, 40)

In [145]:
null_counts = final_df.isnull().sum()
null_counts[null_counts > 0]

country             478   
agent               16262 
company             112218
visibility          6880  
solarenergy         118984
heatindex           86992 
solarradiation      118984
weathertype         118984
snowdepth           92293 
sealevelpressure    38633 
wgust               97714 
windchill           105383
info                118984
dtype: int64

In [136]:
final_df.to_csv('hotel_mod2.csv')

In [137]:
final_df.to_pickle('hotel_mod2.pickle')